In [1]:
import os
import torch
from options import Option
from data_utils.dataset import load_data_test
from model.model import Model
from utils.util import setup_seed, load_checkpoint
import torchvision
import einops

In [2]:
# Set arguments of checkpoint and dataset

args = Option().parse()
args.load = "./checkpoints/sketchy_ext/best_checkpoint.pth"
args.batch = 2

args.valid_shrink_sk=200
args.valid_shrink_im=100

print("test args:", str(args))

import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from utils.ap import calculate
from tqdm import tqdm
import time

os.environ["CUDA_VISIBLE_DEVICES"] = args.choose_cuda
print("current cuda: " + args.choose_cuda)
setup_seed(args.seed)

test args: Namespace(data_path='./datasets', dataset='sketchy_extend', test_class='test_class_sketchy25', cls_number=100, d_model=768, d_ff=1024, head=8, number=1, pretrained=True, anchor_number=49, save='./checkpoints/sketchy_ext', batch=2, epoch=30, datasetLen=10000, learning_rate=1e-05, weight_decay=0.01, pretrain_load=None, load='./checkpoints/sketchy_ext/best_checkpoint.pth', retrieval='rn', testall=False, test_sk=20, test_im=20, num_workers=4, valid_shrink_sk=200, valid_shrink_im=100, choose_cuda='0', seed=2021)
current cuda: 0


In [3]:
# prepare data
sk_valid_data, im_valid_data = load_data_test(args)

# prepare model
model = Model(args)
model = model.half()

if args.load is not None:
    checkpoint = load_checkpoint(args.load)

# load pretrained model
cur = model.state_dict()
new = {k: v for k, v in checkpoint['model'].items() if k in cur.keys()}
cur.update(new)
model.load_state_dict(cur)

if len(args.choose_cuda) > 1:
    model = torch.nn.parallel.DataParallel(model.to('cuda'))
model = model.cuda()

model.eval()
torch.set_grad_enabled(False)

used for valid or test sketch / image:
(77,) (172,)
used for train sketch / image:
(55252,) (68401,)
=> loading model './checkpoints/sketchy_ext/best_checkpoint.pth'


In [4]:
# load data from validset
# 77 sk, 172 im

print('loading image data')
sk_dataload = DataLoader(sk_valid_data, batch_size=args.test_sk, num_workers=args.num_workers, drop_last=False)
print('loading sketch data')
im_dataload = DataLoader(im_valid_data, batch_size=args.test_im, num_workers=args.num_workers, drop_last=False)

loading image data
loading sketch data


In [5]:
# get all sk and im distances, all_dist

dist_im = None # distance items in per for loop
all_dist = None #(77,172)

for i, (sk, sk_label) in enumerate(tqdm(sk_dataload)):
        #sk.shape=(20,3,224,224)
        print(i)
        if i == 0:
            all_sk_label = sk_label.numpy()
        else:
            all_sk_label = np.concatenate((all_sk_label, sk_label.numpy()), axis=0)

        sk_len = sk.size(0)
        sk = sk.cuda()
        #debug
        # print(sk[0].shape)
        # cv2.imwrite(f"./logs/sk-{i}",sk[0].cpu().numpy())
        # if i==0:
        #     grid_sk = torchvision.utils.make_grid(sk)
        #     torchvision.utils.save_image(grid_sk,f"./logs/sk.jpg")
        
        sk, sk_idxs = model(sk, None, 'test', only_sa=True)#sk.shape=(20,192,768)
        for j, (im, im_label) in enumerate(tqdm(im_dataload)):
            if i == 0 and j == 0:
                all_im_label = im_label.numpy()
            elif i == 0 and j > 0:
                all_im_label = np.concatenate((all_im_label, im_label.numpy()), axis=0)

            im_len = im.size(0)
            im = im.cuda()
            im, im_idxs = model(im, None, 'test', only_sa=True)

            sk_temp = sk.unsqueeze(1).repeat(1, im_len, 1, 1).flatten(0, 1).cuda() #(400,197,768) #?difference
            im_temp = im.unsqueeze(0).repeat(sk_len, 1, 1, 1).flatten(0, 1).cuda() #(400,197,768)
            
            if args.retrieval == 'rn':
                feature_1, feature_2 = model(sk_temp, im_temp, 'test')
            #? when retrieval == 'sa'
            if args.retrieval == 'sa':
                feature_1, feature_2 = torch.cat((sk_temp[:, 0], im_temp[:, 0]), dim=0), None

            # print(feature_1.size())    # [2*sk*im, 768] #2 means sk and im cls
            # print(feature_2.size())    # [sk*im, 1]

            if args.retrieval == 'rn':
                if j == 0:
                    dist_im = - feature_2.view(sk_len, im_len).cpu().data.numpy()  # 1*args.batch
                else:
                    dist_im = np.concatenate((dist_im, - feature_2.view(sk_len, im_len).cpu().data.numpy()), axis=1)
            if args.retrieval == 'sa':
                dist_temp = F.pairwise_distance(F.normalize(feature_1[:sk_len * im_len]),
                                                F.normalize(feature_1[sk_len * im_len:]), 2)
                if j == 0:
                    dist_im = dist_temp.view(sk_len, im_len).cpu().data.numpy()
                else:
                    dist_im = np.concatenate((dist_im, dist_temp.view(sk_len, im_len).cpu().data.numpy()), axis=1)

        if i == 0:
            all_dist = dist_im
        else:
            all_dist = np.concatenate((all_dist, dist_im), axis=0)
        print(all_dist.shape)
        #all_dist.shape=(all_sk_label.size, all_im_label.size)
    # print(all_sk_label.size, all_im_label.size)     # [762 x 1711] / 2
class_same = (np.expand_dims(all_sk_label, axis=1) == np.expand_dims(all_im_label, axis=0)) * 1
# print(all_dist.size, class_same.size)     # [762 x 1711] / 2

  0%|          | 0/4 [00:00<?, ?it/s]

0


 25%|██▌       | 1/4 [00:23<01:11, 23.85s/it]

(20, 172)
1


 50%|█████     | 2/4 [00:40<00:39, 19.59s/it]

(40, 172)
2


 75%|███████▌  | 3/4 [00:57<00:18, 18.25s/it]

(60, 172)
3


100%|██████████| 4/4 [01:11<00:00, 17.83s/it]

(77, 172)


In [6]:
# get mAP and Precision

np.savetxt("./logs/all_dist",all_dist)
np.savetxt("./logs/class_same",class_same)

map_all, map_200, precision100, precision200 = calculate(all_dist, class_same, test=True)
print(map_all,map_200,precision100,precision200)
#0.5744169002393182 0.5744169002393182 0.5050711193568337 0.5050711193568337

eval time: 0.5534577369689941
0.5744169002393182 0.5744169002393182 0.5050711193568337 0.5050711193568337


In [7]:
# sort all_dist to get the closest images of the sketches

arg_sort_sim = all_dist.argsort()   # 得到从小到大索引值
print(arg_sort_sim.shape)
print(arg_sort_sim)
np.savetxt("./logs/arg_sort_sim",torch.tensor(arg_sort_sim,dtype=int))

(77, 172)
[[ 28  29  30 ...  59  58  21]
 [108  26   0 ...  21  58   6]
 [  0  30  27 ...   6  58  21]
 ...
 [ 16 120 121 ...  42  62   6]
 [ 20 144 146 ...   6  39   5]
 [ 77  99 145 ...   1  59  86]]


In [8]:
def patch2im(patch_index,im, patch_size):
    '''
    Get the array of the image by the patch index
    
    Args:
        im: (c, h, w)
        patch_index: (2)
        patch: int
    Returns:
        (c, patch_size, patch_size)
    '''
    # print(patch_index.shape, im.shape, patch_size)
    # print(patch_index)
    # print(patch_index[0].item()*patch_size)
    # print(im.shape)
    return im[:, \
        patch_index[0]*patch_size:(patch_index[0].item()+1)*patch_size, \
        patch_index[1].item()*patch_size:(patch_index[1].item()+1)*patch_size]


In [9]:
def patch_replace_data(im_indices, im):
    '''
    Concatenate the target replaced im embedding by the im_indices
    
    Args:
        im_index: (2) [0]->batch_index, [1]->patch_num_index
        im: (b, n, d)
    Returns:
        im_rtn: (len(im_index), d)
    '''
    for i, v in enumerate(im_indices):
        if i == 0:
            print(v)
            im_rtn = im[v[0]][v[1]].unsqueeze(0)
            print(im_rtn.shape)
        else:
            im_rtn = torch.cat((im_rtn, im[v[0]][v[1]].unsqueeze(0)))

    return im_rtn

In [10]:
def patch_replace_rgb(im_indices, im, patch_size):
    '''
    Concatenate the target replaced im embedding by the im_indices
    
    Args:
        im_index: (2) [0]->batch_index, [1]->patch_num_index
        im: (b, c, h, w)
    Returns:
        im_rtn: (len(im_index), d)
    '''
    
    patch_num = im.size(-1) // patch_size
    for i, v in enumerate(im_indices):
        if i == 0:
            print(v, im.shape, im_indices.shape,np.unravel_index(v[1],(patch_num,patch_num)))
            im_rtn = patch2im(np.unravel_index(v[1],(patch_num,patch_num)),im[v[0]],patch_size).unsqueeze(0)
            print(im_rtn.shape)
        else:
            im_rtn = torch.cat((im_rtn, patch2im(np.unravel_index(v[1],(patch_num,patch_num)),im[v[0]],patch_size).unsqueeze(0)))

    return im_rtn

In [11]:
def sk_n_closest(sk_index,sort_sim,n):
    '''
    Args: 
        sk_index: sk index
        sort_sim: sorted similarity matrix,(sk_num, im_num)
        n: the returned number of im
    Returns:
        sk: [1,c,h,w]
        im: [n,c,h,w]
    '''
    im_index = sort_sim[sk_index,:n]
    
    (sk,_) = sk_valid_data[sk_index]
    sk = sk.unsqueeze(0)

    tmp = [im_valid_data[i] for i in im_index]
    im = [i[0].unsqueeze(0) for i in tmp]
    im = torch.concatenate(im)
    # print(sk.shape, im.shape)
    return sk, im

In [79]:
# get the sk RGB and top 3 im RGB which matches the sk

sk_index = 1
sk, im = sk_n_closest(sk_index, arg_sort_sim, 3)
print(sk.shape, im.shape)

torchvision.utils.save_image(sk.cuda(),f"./logs/sk-{sk_index}.jpg")

im_grid = torchvision.utils.make_grid(im,padding=0)
torchvision.utils.save_image(im_grid.cuda(),f"./logs/im_top_{im.size(0)}_of_{sk_index}.jpg")
print(sk.shape, im_grid.shape)

torch.Size([1, 3, 224, 224]) torch.Size([3, 3, 224, 224])
torch.Size([1, 3, 224, 224]) torch.Size([3, 224, 672])


In [27]:
# custom sk and im data

# im_index = 0
# (sk,_) = sk_valid_data[0]
# sk = sk.unsqueeze(0)
# (im,_) = im_valid_data[im_index]
# im = im.unsqueeze(0)
# print(sk.shape,im.shape)

# sk = sk.cuda()
# im = im.cuda()
# print(im.dtype, im.device)

torch.Size([1, 3, 224, 224]) torch.Size([1, 3, 224, 224])
torch.float16 cuda:0


In [69]:
#embed sk and im

from model import rn

#sa
sk_sa, sk_idxs = model(sk.cuda(), None, 'test', only_sa=True)#sk_sa.shape=(20,192,768)
im_sa, im_idxs = model(im.cuda(), None, 'test', only_sa=True)#im_sa.shape=(20,192,768)
sk_im_sa = torch.cat((sk_sa, im_sa), dim=0)
print(sk_im_sa.shape)
print(im_sa, open("./logs/im_sa","w"))
print(im_sa)

#ca
ca_fea = model.ca(sk_im_sa)  # [2b, 197, 768]
cls_fea = ca_fea[:, 0]  # [2b, 1, 768]
token_fea = ca_fea[:, 1:]  # [2b, 196, 768]
print(token_fea.shape)


token_fea_tmp = einops.rearrange(token_fea, "b (h w) c -> b c h w", h=14)
print(token_fea_tmp.shape) #(b, 768, 14,14)
up_fea = model.output4VQGAN(token_fea_tmp)
print(up_fea.shape) #(b, 256,32,32)
up_fea = einops.rearrange(up_fea, "b c h w -> b (h w) c")
print(up_fea.shape) #(b, 1024, 256)

torch.Size([4, 197, 768])
tensor([[[ 4.9316e-01, -4.1290e-02,  8.8745e-02,  ..., -4.6265e-01,
          -5.7129e-01, -2.7441e-01],
         [ 3.3081e-01,  2.9810e-01,  1.7529e-01,  ..., -2.1240e-01,
          -7.2021e-01, -1.8469e-01],
         [ 3.8159e-01,  3.7646e-01,  1.8750e-01,  ..., -2.0508e-01,
          -6.6992e-01, -9.9609e-02],
         ...,
         [ 3.3911e-01,  3.1494e-01,  1.6821e-01,  ..., -1.8494e-01,
          -7.1045e-01, -1.5942e-01],
         [ 3.5474e-01,  3.3350e-01,  1.5076e-01,  ..., -1.9910e-01,
          -6.8750e-01, -1.3074e-01],
         [ 3.2739e-01,  3.2153e-01,  1.5869e-01,  ..., -2.4011e-01,
          -7.0508e-01, -1.4417e-01]],

        [[ 7.6318e-01,  4.7729e-02, -1.0077e-01,  ..., -3.8965e-01,
          -4.0723e-01,  7.7332e-02],
         [ 5.0342e-01,  1.7859e-01, -1.7236e-01,  ..., -3.3008e-01,
          -6.8506e-01, -7.6965e-02],
         [ 5.1904e-01,  2.1741e-01, -4.7394e-02,  ..., -4.4385e-01,
          -5.8691e-01, -2.1637e-02],
         ...,

In [83]:
# select patch_size 7-32*32 or 16-14*14

# sk_fea = up_fea[0]
# im_fea = up_fea[sk.size(0):]

sk_fea = token_fea_tmp[0]
sk_fea = sk_fea.unsqueeze(0)
im_fea = token_fea_tmp[sk.size(0):]
print(sk_fea.shape,im_fea.shape)

sk_fea = einops.rearrange(sk_fea, "b c h w -> b (h w) c")
im_fea = einops.rearrange(im_fea, "b c h w -> b (h w) c")

print(sk_fea.shape,im_fea.shape) #(b, 1024, 256)

torch.Size([1, 768, 14, 14]) torch.Size([3, 768, 14, 14])
torch.Size([1, 196, 768]) torch.Size([3, 196, 768])


In [84]:
# calculate the cos_scores between sk_fea and n im_fea
# cos_scores (n,1024,1024)

# np.savetxt("./logs/sk_fea", sk_fea.cpu())
# np.savetxt("./logs/im_fea", im_fea.cpu())
print(sk_fea.shape, im_fea.shape)
cos_scores = rn.cos_similar(sk_fea, im_fea)
print(cos_scores.shape)
np.savetxt("./logs/cos_scores",cos_scores.cpu()[0])

torch.Size([1, 196, 768]) torch.Size([3, 196, 768])
torch.Size([3, 196, 196])


In [85]:
# get the max_indices of im_fea to be replaced

max_indices = torch.empty((0,2), dtype=int)

b = einops.rearrange(cos_scores,"n s i -> s (n i)") # 1024 x 3072
print(b)


for i in b:
    max_indices_item = torch.argmax(i)
    # print(i.shape)
    new = np.unravel_index(max_indices_item.cpu(),(cos_scores.shape[0],cos_scores.shape[2]))
    # print(torch.Tensor(new))
    max_indices = torch.cat((max_indices, torch.tensor(new, dtype=torch.int).unsqueeze(0)), 0)
    # print(max_indices)
    
# print(np.unravel_index(b.values, (3, 196)))
np.savetxt("./logs/max_indices",max_indices)
print(max_indices.shape)


tensor([[0.7505, 0.7676, 0.7593,  ..., 0.6045, 0.6123, 0.5986],
        [0.7598, 0.7744, 0.7681,  ..., 0.6001, 0.6064, 0.5957],
        [0.7612, 0.7695, 0.7671,  ..., 0.5977, 0.6011, 0.5938],
        ...,
        [0.7544, 0.7646, 0.7607,  ..., 0.6001, 0.6045, 0.5942],
        [0.7476, 0.7637, 0.7563,  ..., 0.6045, 0.6113, 0.5986],
        [0.7314, 0.7534, 0.7432,  ..., 0.6084, 0.6162, 0.6035]],
       device='cuda:0', dtype=torch.float16)
torch.Size([196, 2])


In [32]:
# replace the sk embeddings with im embeddings

# im_replaced = patch_replace_data(max_indices, im_fea)
# print(im_replaced.shape)

tensor([ 0, 14])
torch.Size([1, 768])
torch.Size([196, 768])


In [86]:
print(im.shape)
# im = im.squeeze()
print(max_indices.shape, max_indices.dtype)

torch.Size([3, 3, 224, 224])
torch.Size([196, 2]) torch.int64


In [87]:
im_replaced = patch_replace_rgb(max_indices, im,16)
print(im_replaced.shape)

tensor([ 0, 14]) torch.Size([3, 3, 224, 224]) torch.Size([196, 2]) (1, 0)
torch.Size([3, 224, 224])
torch.Size([1, 3, 16, 16])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 

In [88]:
im_replaced = torchvision.utils.make_grid(im_replaced,nrow=14,padding=0)
print(im_replaced.shape)
torchvision.utils.save_image(im_replaced.cuda(),"./logs/patch_replace_14_top3.jpg")

torch.Size([3, 224, 224])


In [ ]:
# test patch_replace_data function
# test = torch.arange(0,3*1024).reshape(3,1024,1)
# im_replaced_test = patch_replace_data(max_indices, test)
# print(im_replaced_test, file=open("logs/im_replaced_rgb","w"))

In [ ]:
# configure vqgan model

import torch
vqgan_dict = torch.load("../download/last.ckpt")

# !proxychains git clone https://github.com/CompVis/taming-transformers
# %cd taming-transformers

# !mkdir -p logs/vqgan_imagenet_f16_1024/checkpoints
# !mkdir -p logs/vqgan_imagenet_f16_1024/configs
# # !wget 'https://heibox.uni-heidelberg.de/f/140747ba53464f49b476/?dl=1' -O 'logs/vqgan_imagenet_f16_1024/checkpoints/last.ckpt' 
# !proxychains wget 'https://heibox.uni-heidelberg.de/f/6ecf2af6c658432c8298/?dl=1' -O 'taming-transformers/logs/vqgan_imagenet_f16_1024/configs/model.yaml' 

# %pip install omegaconf>=2.0.0 pytorch-lightning>=1.0.8 einops>=0.3.0

import sys
sys.path.append("taming-transformers/")

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [ ]:
# def vqgan load function and scripts

import yaml
import torch
from omegaconf import OmegaConf
from taming.models.vqgan import VQModel, GumbelVQ

def load_config(config_path, display=False):
  config = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config)))
  return config

def load_vqgan(config, ckpt_path=None, is_gumbel=False):
  if is_gumbel:
    model = GumbelVQ(**config.model.params)
  else:
    model = VQModel(**config.model.params)
  if ckpt_path is not None:
    sd = torch.load(ckpt_path, map_location="cpu")["state_dict"]
    missing, unexpected = model.load_state_dict(sd, strict=False)
  return model.eval()


In [ ]:
# load vqgan model as model1024

config1024 = load_config("taming-transformers/logs/vqgan_imagenet_f16_1024/configs/model.yaml", display=False)
model1024 = load_vqgan(config1024, ckpt_path="taming-transformers/logs/vqgan_imagenet_f16_1024/checkpoints/last.ckpt").to(DEVICE)

print(model1024)
print(model1024, file=open("logs/model1024_info","w"))

In [ ]:
# rearrange the im_replaced up_fea to match the input of vqgan quantize

h = einops.rearrange(im_fea,"b (h w) c -> b c h w",h=32) #(1,256,1024)
h = h.to(torch.float32)
h = h *10
# h = h.flatten()
print(h, file=open("./logs/h","w"))
print(h.shape)
print(h.dtype)

In [ ]:
# quantize

quant, emb_loss, info = model1024.quantize(h) #don't use same name
print(quant, emb_loss, info)

In [ ]:
#decode

dec = model1024.decoder(quant)
print(dec.shape)
torchvision.utils.save_image(dec,"logs/dec.jpg")

Test Model1024

In [ ]:
# test model1024

im_index = 28
(sk,_) = sk_valid_data[0]
(im,_) = im_valid_data[im_index]
im = im.unsqueeze(0)
print(sk.shape,im.shape)

im = im.to(torch.float32).cuda()
print(im.dtype, im.device)


In [ ]:
im = im.to(torch.float32)
quant, emb_loss, info = model1024.encode(im)

h = model1024.encoder(im)
h = model1024.quant_conv(h)
print(h,file=open("./logs/im_embeded","w"))

dec = model1024.decode(quant)

print(im_index)
torchvision.utils.save_image(im,f"./logs/im_test{im_index}_2.jpg")
torchvision.utils.save_image(dec,f"./logs/dec_test{im_index}_2.jpg")